In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [2]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
# fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = cohort.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                   Slp filename Start time
0           0       2     Day  2020_07_20_09_53_59_813912_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_09_53_59_813912_compressed_Day.slp   09:53:59
1           1       2     Day  2020_07_20_10_16_08_629297_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_16_08_629297_compressed_Day.slp   10:16:08
2           2       2     Day  2020_07_20_10_38_20_043555_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_10_38_20_043555_compressed_Day.slp   10:38:20
3           3       2     Day  2020_07_20_11_00_31_566455_compressed.mp4     P15             6                  NaN        1           1  2020_07_20_11_00_31_566455_compressed_Day.slp   11:00:31
4           4       

In [3]:
#################################################
########### COMPUTE HUDDLE COMPOSITION ##########
#################################################

#
cohort.video_frame_rate = 24

# set the flags for the huddle corrections
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.load_huddle_tracks()

# # this just loads the _spine.npy files;
cohort.use_nohuddle = True
cohort.load_feature_tracks()

# # this computes mode-based occupancy in the nest with 1min bins
cohort.median_filter_width = cohort.video_frame_rate*5+1 # 1: there is no smotthing; n x frame rate gives you n seconds of somothing (always add 1 to get odd number)
cohort.n_frames_per_bin = cohort.video_frame_rate  # single second resolution
cohort.n_cores = 24
cohort.compute_huddle_composition()
print ("# of huddle composition arrays: ", len(cohort.huddle_comps_binned), cohort.huddle_comps_binned[0].shape)


 # of files:  991 example:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/huddles/2020_07_20_09_53_59_813912_compressed_Day_huddle_spine_fixed_interpolated.npy
# of feature tracks:  991


1001it [02:43,  6.14it/s]                                                       


# of huddle composition arrays:  991 (6, 1201)


In [44]:
#################################################
########### DISPLAY HUDDLE COMPOSITION ##########
#################################################

#
cohort.forward_padding = 20 # padding in percent
cohort.generate_huddle_composition_ethograms()

#


size of img:  (86400, 96)  size of flatten image:  (1382400, 6)


100%|███████████████████████████████████████| 991/991 [00:00<00:00, 2270.41it/s]
